# Hurdle 2
## Introduction

DARPA will score all Hurdle 2 submissions by running participant provided code against DARPA provided test stimulus and scoring code. Both participant code and DARPA provided code will be encapsulated in the LXC container format. See here for an introduction to LXC:

https://linuxcontainers.org/lxc/introduction/  

This repository includes a build script for generating the DARPA Scoring Container and an example Solution Container. The **Scoring Container** is provided by DARPA and includes the test stimulus and scoring code. The **Solution Container** includes a stub solution which shows an example of how to interact with the Scoring Container. 

When evaluating whether a participant's submitted solution passes or fails Hurdle 2, DARPA will use a Scoring Container built using the build_hurdle_2_trusty.sh script provided in this directory. The only difference will be in the particular bandplan used to generate the combined I/Q test file.

## Environment
See the document entitled LXC_Configuration_Instructions.ipynb at the root of the Phase 1 Hurdles repository for instructions on how to configure your Ubuntu host to run LXC containers.


## Building the LXC Containers

To build both the Scoring Container and the example Solution Container for Hurdle 3, run the following command from inside your cloned copy of the phase1-hurdles repository:

```console
user@host:~/phase1-hurdles/hurdle2$ ./build_hurdle_2_trusty.sh
```

This script will create a new lxc subdirectory in your current directory, containing two containers. Assuming the build script was run from ~/phase1-hurdles/hurdle2, there will be containers in:

~/phase1-hurdles/hurdle2/lxc/hurdle2_scoring_trusty_lxc 

and:

~/phase1-hurdles/hurdle3/lxc/hurdle2_solution_trusty_lxc. 

Note that this will build GNU Radio and UHD from source and may take a considerable amount of time, depending on the hardware you are using. Build times above 20 minutes are not uncommon.

To run any of the standard lxc- commands such as lxc-attach, you will need to specify the absolute path to the lxc directory as the -P prefix parameter. 

## Starting the containers

Before you can use the containers, you must start each of them. 

The -P parameter specifies where LXC will look for containers. -n specifies the name of the container. This example assumes you have the containers in ~/phase1-hurdles/hurdle2/lxc and are currently in the ~/phase1-hurdles/hurdle2 directory. To run both containers, use the following commands:

First store the LXC path to a shell variable:

```console
user@host:~/phase1-hurdles/hurdle2$ LXC_PATH=$(pwd)/lxc
```

Then start both containers:


```console
user@host:~/phase1-hurdles/hurdle2$ lxc-start -P "$LXC_PATH" -n hurdle2_scoring_trusty_lxc
user@host:~/phase1-hurdles/hurdle2$ lxc-start -P "$LXC_PATH" -n hurdle2_solution_trusty_lxc
```

## Running the example hurdle solution against the DARPA scoring container

You can SSH into each container now that they are both running. To get the IP address of the scoring container, use 

```console
user@host:~/phase1-hurdles/hurdle2$ lxc-info -P "$LXC_PATH" -n hurdle2_scoring_trusty_lxc -iH
```

For this example, lets assume that command returned 10.0.3.132. 

To get the IP address of the solution container, use

```console
user@host:~/phase1-hurdles/hurdle2$ lxc-info -P "$LXC_PATH" -n hurdle2_solution_trusty_lxc -iH
```

For this example, lets assume that command returned 10.0.3.147. 

Our scoring container is then running at 10.0.3.132, and our solution container is running at 10.0.3.147. 


SSH into the **scoring** container as the user sc2 with:

```console
user@host:~/phase1-hurdles/hurdle2$ ssh sc2@10.0.3.132
```

If this is the first time connecting to the container, you'll see a message similar to: 

```console
The authenticity of host '10.0.3.132 (10.0.3.132)' can't be established.
ECDSA key fingerprint is SHA256:5vkTtMcNxVQPKhTuIZhifJrwxWoUCVqmguvl6Ot4ZV0.
Are you sure you want to continue connecting (yes/no)? 
```

Answer **yes**. 


You'll then be prompted for a password. Use: **sc2**


You should then see: 
```console
Welcome to Ubuntu 14.04.5 LTS (GNU/Linux 4.4.0-38-generic x86_64)

 * Documentation:  https://help.ubuntu.com/

The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.

sc2@hurdle2_scoring_trusty_lxc:~$ 
```

Change to the hurdle 2 directory and generate a new band plan, and start the scoring application. Use the following commands:

```console
sc2@hurdle2_scoring_trusty_lxc:~$ cd ~/phase1-hurdles/hurdle2
sc2@hurdle2_scoring_trusty_lxc:~/phase1-hurdles/hurdle2$ ./generate_band_plan.py
sc2@hurdle2_scoring_trusty_lxc:~/phase1-hurdles/hurdle2$ ./run_hurdle2.py --bandplan=band_data.json
```

In a new terminal prompt on your host machine, SSH to the **solution** container as user sc2 with:

```console
user@host:~/phase1-hurdles/hurdle2$ ssh sc2@10.0.3.147
```

If this is the first time connecting to the container, you'll see a message similar to: 

```console
The authenticity of host '10.0.3.147 (10.0.3.147)' can't be established.
ECDSA key fingerprint is SHA256:5vkTtMcNxVQPKhTuIZhifJrwxWoUCVqmguvl6Ot4ZV0.
Are you sure you want to continue connecting (yes/no)?
```

Answer **yes**. 


You'll then be prompted for a password. Use: **sc2**


You should then see: 
```console
Welcome to Ubuntu 14.04.5 LTS (GNU/Linux 4.4.0-38-generic x86_64)

 * Documentation:  https://help.ubuntu.com/

The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.

sc2@hurdle2_solution_trusty_lxc:~$
```

Change to the hurdle 2 directory and start the solution application using the following commands:
Note that you will need to tell the solution application the IP address of the **Scoring Container**, in this case 10.0.3.132. 

```console
sc2@hurdle2_solution_trusty_lxc:~$ cd ~/phase1-hurdles/hurdle2
sc2@hurdle2_solution_trusty_lxc:~/phase1-hurdles/hurdle2$ ./hurdle2_solution_stub_edited.py --hurdle2-scoring-host=10.0.3.132
```

Currently the application prints out nothing, and actually hangs on exit. This appears to be a bug in the TCP blocks used by GNU Radio not shutting down correctly. Give this block a minute to run and kill it with CTRL-C. 

The scoring application should print out something along the lines of

```console
Received Answer: {0: 0, 1: 0, 2: 0, ....
Converted Answer to: {0: 'NOISE', 1: 'NOISE', 2: 'NOISE', ....
Minimum score needed: 0.68 Score received: -0.125
Hurdle 2 Failed
Detailed Results: {'Pfa': 0.7142857142857143, 'num_false_alarms': 10, 'score': -0.125, 'num_occupied_bins': 16, 'Pd': 0.6875, 'hurdle_pass': False, 'num_detections': 11, 'num_wrong_type_reports': 18, 'num_bins': 30, 'Pt': -0.6363636363636364}
Scoring done.
Scoring process join complete.
```

after you kill the example solution.

You may also need to force the scoring application to quit using CTRL-C